In [1]:
from tensorflow.python.compiler.tensorrt.trt_convert import TrtPrecisionMode

In [2]:
model_path = None
rtype = "panda"
PRECISION = TrtPrecisionMode.FP32

In [3]:
import SharedArray as sa
import numpy as np
import time
import os
import sys
import shutil
import random
import time

PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
sys.path.append(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.controller.robot_config import RobotType
from pkg.planning.filtering.lattice_model.data_utils import *
import numpy as np
int2rtypename = {v.value:v.name for v in RobotType}
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

In [4]:
ROBOT_TYPE_NAME = rtype
ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
last_save = sorted([item for item in os.listdir(os.path.join(ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
model_path_rel = os.path.join(last_model, last_save)
model_log_dir = os.path.join(ROBOT_MODEL_ROOT, model_path_rel)
model_log_dir_trt = os.path.join(ROBOT_MODEL_ROOT, model_path_rel.replace("model", "trt")+"-"+PRECISION)

In [5]:
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")

ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)

dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

N_max_train = 10000
N_max_test = 5000

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))

['20210920-194311', '20210920-233941', '20210921-032259', '20210921-072543', '20210921-111615', '20210921-150638', '20210921-192001', '20210921-232015', '20210922-033134', '20210922-074138']
['20210922-113744', '20210922-153855', '20210922-193840', '20210922-233844', '20210923-040005']
train set: 10003
test set: 5000
filtered data: 10000 / 5000


# Convert and save frozen graph

In [6]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
model = tf.keras.models.load_model(model_log_dir)

In [8]:
import onnx
temp_onnx_dir = os.path.join(ROBOT_MODEL_ROOT, 
                                  model_path_rel.split("/")[0], "temp.onnx")

In [9]:
    print("!python3 -m tf2onnx.convert --saved-model {} --output {}".format(model_log_dir, temp_onnx_dir))

!python3 -m tf2onnx.convert --saved-model /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/model_17 --output /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/temp.onnx


### convert tf2onnx - copy and run result of above cell

In [10]:
!python3 -m tf2onnx.convert --saved-model /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/model_17 --output /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/temp.onnx

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-04 19:51:19,094 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-10-04 19:51:19,412 - WARNING - Importing a function (__inference_dense_bn_layer_call_and_return_conditional_losses_1238642) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2021-10-04 19:51:19,458 - WARNING - Importing a function (__inference_res_net_model_tp_layer_call_and_return_conditional_losses_1226173) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2021-10-04 19:51:19,825 - WARNING - Importing a function (__inference_res_net_model_tp_layer_call_and_return_conditional_losses_1231329) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2021-10-04

2021-10-04 19:51:39,068 - WARNING - Failed to apply optimize_transpose
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python3.6/site-packages/tf2onnx/optimizer/__init__.py", line 62, in optimize_graph
    graph = opt.optimize(current, iteration) or graph
  File "/home/rnb/.local/lib/python3.6/site-packages/tf2onnx/optimizer/optimizer_base.py", line 41, in optimize
    graph = self._optimize(graph)
  File "/home/rnb/.local/lib/python3.6/site-packages/tf2onnx/optimizer/transpose_optimizer.py", line 157, in _optimize
    return self._apply_optimization(graph, self._optimize_at_current_graph_level)
  File "/home/rnb/.local/lib/python3.6/site-packages/tf2onnx/optimizer/optimizer_base.py", line 62, in _apply_optimization
    graph = optimize_func(graph)
  File "/home/rnb/.local/lib/python3.6/site-packages/tf2onnx/optimizer/transpose_optimizer.py", line 191, in _optimize_at_current_graph_level
    self.post_optimize_action()
  File "/home/rnb/.local/lib/python3.6/site-package

In [11]:
onnx_model  = onnx.load_model(temp_onnx_dir)

In [12]:
BATCH_SIZE = 1
inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = BATCH_SIZE

In [13]:
onnx_model_path = model_log_dir.replace("model_", "onnx_")+".onnx"
trt_model_path = model_log_dir.replace("model_", "trt_engine_")+".trt"
onnx.save_model(onnx_model, onnx_model_path)

In [14]:
print("!trtexec --onnx={} --saveEngine={}  --explicitBatch".format(onnx_model_path, trt_model_path))

!trtexec --onnx=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/onnx_17.onnx --saveEngine=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/trt_engine_17.trt  --explicitBatch


### convert onnx2trt - copy and run result of above cell

In [15]:
!trtexec --onnx=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/onnx_17.onnx --saveEngine=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/trt_engine_17.trt  --explicitBatch

&&&& RUNNING TensorRT.trtexec [TensorRT v8003] # trtexec --onnx=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/onnx_17.onnx --saveEngine=/home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/trt_engine_17.trt --explicitBatch
[10/04/2021-19:52:28] [I] === Model Options ===
[10/04/2021-19:52:28] [I] Format: ONNX
[10/04/2021-19:52:28] [I] Model: /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/onnx_17.onnx
[10/04/2021-19:52:28] [I] Output:
[10/04/2021-19:52:28] [I] === Build Options ===
[10/04/2021-19:52:28] [I] Max batch: explicit
[10/04/2021-19:52:28] [I] Workspace: 16 MiB
[10/04/2021-19:52:28] [I] minTiming: 1
[10/04/2021-19:52:28] [I] avgTiming: 8
[10/04/2021-19:52:28] [I] Precision: FP32
[10/04/2021-19:52:28] [I] Calibration: 
[10/04/2021-19:52:28] [I] Refit: Disabled
[10/04/2021-19:52:28] [I] Sparsity: Disabled
[10/04/2021-19:52:28] [I] Safe mode: Disabled
[10/04/2021-19:52:28] [I] Restricted mode: Disabled
[10/04/2021-1

In [6]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
def my_input_fn():
    data_pairs_train = data_pairs_train_filtered
    data_batch = []
    i_step = 0
    for data_pair in data_pairs_train:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        data_batch.append([grasp_img, arm_img, rh_mask])
        if i_step%BATCH_SIZE==0:
            grasp_img_batch = tf.constant([grasp_img for grasp_img, arm_img, rh_mask in data_batch], tf.float32)
            arm_img_batch = tf.constant([arm_img for grasp_img, arm_img, rh_mask in data_batch], tf.float32)
            rh_mask_batch = tf.constant([rh_mask for grasp_img, arm_img, rh_mask in data_batch], tf.float32)
            data_batch = []
            yield grasp_img_batch, arm_img_batch, rh_mask_batch

In [8]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(precision_mode=PRECISION)

converter = trt.TrtGraphConverterV2(input_saved_model_dir=model_log_dir,
                                    conversion_params=conversion_params)
if PRECISION == "INT8":
    converter.convert(calibration_input_fn=my_input_fn)
else:
    converter.convert()

INFO:tensorflow:Linked TensorRT version: (7, 2, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 3)


In [9]:
converter.build(input_fn=my_input_fn)

In [10]:
converter.save(model_log_dir_trt)

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/trt_17-FP32/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda/20211003-081015/trt_17-FP32/assets


# Test frozen graph

In [ ]:
grasp_img_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
prepared_p = sa.create(f"shm://{rtype}.prepared", (1,), dtype=np.bool)

grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
prepared_p[0] = True

### Call checker once to get data example. run below cell to return response so the checker can stop waiting

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

saved_model_loaded = tf.saved_model.load(
    model_log_dir_trt, tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[
    signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
frozen_func = convert_variables_to_constants_v2(graph_func)

In [ ]:
query_in[0] = False
grasp_img_t = tf.constant(grasp_img_p, dtype=tf.float32)
arm_img_t = tf.constant(arm_img_p, dtype=tf.float32)
rh_mask_t = tf.constant(rh_mask, dtype=tf.float32)
input_data = (grasp_img_t, arm_img_t, rh_mask_t)
output = frozen_func(*input_data)[0].numpy()    
for i_b in range(BATCH_SIZE):
    result_p[i_b] = output[i_b]
response_out[0] = True

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
while not query_quit[0]:
    while not query_in[0]:
        time.sleep(SERVER_PERIOD)
    query_in[0] = False
    with gtimer.block("frozen_func"):
        grasp_img_t = tf.constant(grasp_img_p, dtype=tf.float32)
        arm_img_t = tf.constant(arm_img_p, dtype=tf.float32)
        rh_mask_t = tf.constant(rh_mask, dtype=tf.float32)
        input_data = (grasp_img_t, arm_img_t, rh_mask_t)
        output = frozen_func(*input_data)[0].numpy()    
        for i_b in range(BATCH_SIZE):
            result_p[i_b] = output[i_b]
    response_out[0] = True
print(gtimer)

In [ ]:
sa.delete(f"shm://{ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.prepared")